# Becoming a BackProp Ninja

In [8]:
import torch
import torch.nn.functional as F

import random
from typing import Any

import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [9]:
# Hyperparaeters
block_size = 5          # Context Length! How many characters to take to predict the next one
embid_size = 15          # How many dimension in the embedding space
batch_size = 256
num_hidden = 256        # Number of neurons in the hidden layer
lr = 0.05
epochs = 5000

In [10]:
# Dataset

DATA_DIR = "../data/indian_names.txt"

train_split = 0.8
dev_split = 0.1
test_split = 0.1

assert (train_split + dev_split + test_split) == 1.0, "Data Split ration does not add up to 1.0"

# Loading data
words = open(DATA_DIR, 'r').read().splitlines()
print(len(words), "words.")

# Building Vocabulary and Conversions   str <--> int
chrs = sorted(list(set(''.join(words))))
stoi = {s : i+1 for i, s in enumerate(chrs)}
itos = {i+1 : s for i, s in enumerate(chrs)}

stoi['.'] = 0
itos[0] = '.'

special_characters = ['.']
chrs = sorted(chrs + special_characters)

# Takes some words and builds X and Y data out of it
def build_dataset(words, type = 'train'):
    
    X, Y = [], []

    for word in words:
        #print(''.join(word))
        context = [0] * block_size
        for ch in word + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(f"Shape of {type} data is     X -> {X.shape}      Y ->  {Y.shape}")

    return X, Y

# Shuffling the data
random.seed(800)
random.shuffle(words)

n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

# Calling build
Xtrain, Ytrain = build_dataset(words[:n1], "train")
Xdev, Ydev = build_dataset(words[n1:n2], "dev")
Xtest, Ytest = build_dataset(words[n2:], "test")

6474 words.
Shape of train data is     X -> torch.Size([38221, 5])      Y ->  torch.Size([38221])
Shape of dev data is     X -> torch.Size([4691, 5])      Y ->  torch.Size([4691])
Shape of test data is     X -> torch.Size([4705, 5])      Y ->  torch.Size([4705])


In [11]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [12]:
# Weights initializing
g = torch.Generator().manual_seed(800)

C = torch.randn((len(chrs), embid_size), generator=g) * 0.01

w1 = torch.randn((block_size*embid_size, num_hidden), generator=g) * (5/3)/((block_size*embid_size)**0.5)
b1 = torch.randn((num_hidden), generator=g) * 0.1

bngain = torch.ones((1, num_hidden)) * 0.1 + 0.1
bnbias = torch.zeros((1, num_hidden)) * 0.1

w2 = torch.randn((num_hidden, len(chrs)), generator=g) * 0.1
b2 = torch.randn(len(chrs), generator=g) * 0.1

parameters = [C, w1, w2, b2, bngain, bnbias]

print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

27056


In [13]:
n = batch_size

# construct a minibatch
ix = torch.randint(0, Xtrain.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtrain[ix], Ytrain[ix] # batch X,Y

In [14]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ w1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ w2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3380, grad_fn=<NegBackward0>)

In [25]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

# dlogprobs = torch.zeros_like(logprobs)
# dlogprobs[range(n), Yb] = -1.0/n
# dprobs = (1.0 / probs) * dlogprobs
# dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
# dcounts = counts_sum_inv * dprobs
# dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
# dcounts += torch.ones_like(counts) * dcounts_sum
# dnorm_logits = counts * dcounts
# dlogits = dnorm_logits.clone()
# dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)
# dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes
# dh = dlogits @ w2.T
# dW2 = h.T @ dlogits
# db2 = dlogits.sum(0)
# dhpreact = (1.0 - h**2) * dh
# dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
# dbnraw = bngain * dhpreact
# dbnbias = dhpreact.sum(0, keepdim=True)
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)
# dembcat = dhprebn @ w1.T
# dW1 = embcat.T @ dhprebn
# db1 = dhprebn.sum(0)
# demb = dembcat.view(emb.shape)
# dC = torch.zeros_like(C)
# for k in range(Xb.shape[0]):
#   for j in range(Xb.shape[1]):
#     ix = Xb[k,j]
#     dC[ix] += demb[k,j]



#  lp = -(a + b + c) / 3     -> lp = -1/3a - 13/b -1/3c     -> dlpa/da = -1/3
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1/n

# l = probs.log()   ->  dl/dprobs = 1/probs
dprobs = torch.ones_like(probs) * (1.0/probs) * dlogprobs

dcounts_sum_inv = counts * dprobs



    
cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
# cmp('counts_sum', dcounts_sum, counts_sum)
# cmp('counts', dcounts, counts)
# cmp('norm_logits', dnorm_logits, norm_logits)
# cmp('logit_maxes', dlogit_maxes, logit_maxes)
# cmp('logits', dlogits, logits)
# cmp('h', dh, h)
# cmp('W2', dW2, w2)
# cmp('b2', db2, b2)
# cmp('hpreact', dhpreact, hpreact)
# cmp('bngain', dbngain, bngain)
# cmp('bnbias', dbnbias, bnbias)
# cmp('bnraw', dbnraw, bnraw)
# cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
# cmp('bnvar', dbnvar, bnvar)
# cmp('bndiff2', dbndiff2, bndiff2)
# cmp('bndiff', dbndiff, bndiff)
# cmp('bnmeani', dbnmeani, bnmeani)
# cmp('hprebn', dhprebn, hprebn)
# cmp('embcat', dembcat, embcat)
# cmp('W1', dW1, w1)
# cmp('b1', db1, b1)
# cmp('emb', demb, emb)
# cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: False | approximate: False | maxdiff: 0.0806659609079361


In [31]:
(counts * counts_sum_inv).shape


torch.Size([256, 27])

In [33]:
counts_sum_inv.shape

torch.Size([256, 1])